In [1]:
import pygame
import itertools
import random
import numpy as np
import sys
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
### unit converters ###

def cm(Input):
    # converts cm to pixels
    return Input/18.0

def m(Input):
    # converts m to pixels
    return cm(Input)/100

def kmh(Input):
    # converts km/h to pixels per frame
    return Input/25.92

def sec(Input):
    # converts seconds to frames
    return Input*40

In [3]:
### Global constants ###

H = 900
B = 250
L = 50

FPS = 60

MAX_SPEED = kmh(50)
FOLLOWING_TIME = sec(1)
BUFFER = 20

# Spawn parameters
SPAWN_RATE = 30 # spawns per minute

# Dimensions
CAR_WIDTH = cm(180)
CAR_LENGTH = (20,27)
TRUCK_LENGTH = (42,50)

TRUCK_PROBABILITY = 0.1
ACCELERATION = 0.02
DECELERATION = -0.02

LEFT = 1

# colors
WHITE = (240, 240, 245)
RED1 = (255, 100, 100)
RED2 = (220, 65, 65)
GREY1 = (150, 150, 150)

In [4]:
class Car():
    def __init__(self, length, path, lights):
        self.length = length
        self.sections, self.blockades = path
        self.i = 0
        self.distance = 0
        self.speed = MAX_SPEED
        self.acceleration = 0
        self.special_priority = False
        self.color = pygame.Color(0)
        self.color.hsla = (random.randint(0,360), 75, 25)
        self.t = 0
        self.lights = lights
        
    def section(self):
        return self.sections[self.i]
    
    def position(self):
        return self.section().position(self.distance)
    
    def angle(self):
        return self.section().angle(self.distance)
    
    def find_next_car(self):
        for car in reversed(self.section().cars):
            if car.distance > self.distance:
                return car
        for section in self.sections[self.i+1:]:
            if section.cars:
                return section.cars[-1]
            
    def update_acceleration(self):
        next_car = self.find_next_car()
        self.acceleration = ACCELERATION
        
        if self.i == 1 and ((self.lights and not self.section().priority) or \
        (not self.lights and any(b.cars for b in self.blockades) and not self.special_priority)):
            space = self.sections[1].length - self.distance - self.length/2
            braking_distance = self.speed**2 / (2 * -DECELERATION)
            difference = space - braking_distance
            if 0 < difference < 10:
                self.acceleration = DECELERATION

        if next_car:
            space = next_car.distance - self.distance - \
                    (self.length + next_car.length)/2 - BUFFER
            
            for i in range(self.i, len(self.sections)):
                if self.sections[i] == next_car.section():
                    break
                space += self.sections[i].length
            
            acceleration = 2*(space - self.speed*FOLLOWING_TIME) / FOLLOWING_TIME**2
            if acceleration < self.acceleration:
                self.acceleration = acceleration

    def update_speed(self):
        self.speed = round(max(0, min(MAX_SPEED, self.speed + self.acceleration)),2)

    def update_distance(self):
        self.distance += self.speed
    
    def update_section(self):
        section = self.section()

        if self.distance > section.length:
            self.distance -= section.length
            section.cars.remove(self)
            self.i += 1
            
            if self.i < len(self.sections):
                self.section().cars.append(self)
            else:
                return True
                
    def update(self):
        self.t += 1
        self.update_acceleration()
        self.update_speed()
        self.update_distance()
        return self.update_section()
        

In [5]:
class Section():
    def __init__(self, x, y, a, b, length):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        self.length = length
        self.cars = []
        self.priority = False

In [6]:
class Line(Section):
    def position(self, s):
        return (self.x + self.a*s, self.y + self.b*s)

    def angle(self, s):
        return np.arctan2(-self.b, self.a)

In [7]:
class Curve(Section):
    def __init__(self, x, y, a, b, l, r):
        super().__init__(x, y, a, b, l*np.pi*r/2)
        self.l = l
        self.r = r

    def position(self, s):
        c = self.a*np.pi/2 + self.b*s/self.r
        return (self.x+self.r*np.cos(c), self.y-self.r*np.sin(c))
    
    def angle(self, s):
        return (self.a+1)*np.pi/2 + self.b*s/self.r

In [8]:
class Intersection():
    def __init__(self, background, sections, paths, triggers, lights=False, spawn_rate=SPAWN_RATE):
        if background:
            self.background = pygame.image.load(background + '.png').convert()
        self.sections = sections
        self.paths = paths
        self.triggers = triggers
        self.car_count = 0
        self.spawn_rate = spawn_rate
        self.cars_per_min = 0
        self.car_time_list = []
        self.average_travel_time = 0
        self.timer = 0
        self.lights = lights
    
    def cars(self):
        for section in self.sections:
            for car in section.cars:
                yield car
        
    def spawn_cars(self, t):
        if not t % int(FPS/(self.spawn_rate/60)):
            if random.random() < TRUCK_PROBABILITY:
                length = random.randint(*TRUCK_LENGTH)
            else:
                length = random.randint(*CAR_LENGTH)

            path = random.choice(self.paths)
            section0 = path[0][0]
            
            if not section0.cars or section0.cars[-1].distance > 80: 
                car = Car(length, path, self.lights)
                section0.cars.append(car)

    def update_cars(self, t):
        if self.lights:
            for s in self.triggers:
                if s.priority and not s.cars:
                    s.priority = False
                    self.timer = t

            if t - self.timer > 60:
                if not any(s.priority for s in self.triggers):
                    occupied = [s for s in self.triggers if s.cars]
                    if occupied:
                        random.choice(occupied).priority = True
        else:
            if not t%60 and not any(s.cars and s.cars[0].speed for s in self.triggers):
                cars = [s.cars[0] for s in self.triggers if s.cars]
                if cars:
                    random.choice(cars).special_priority = True
            
        for section in self.sections:
            for car in section.cars:
                if car.update():
                    self.current_time = pygame.time.get_ticks()/1000
                    self.current_second = round(self.current_time, 1)
                    
                    self.car_count += 1
                    # update cars per minute
                    self.cars_per_min = self.car_count/(t/FPS/60)
                    # update travel times
                    self.car_time_list.append(round(car.t/FPS,3))
                    self.average_travel_time = np.mean(self.car_time_list)

            
    def update(self, t):
        self.spawn_cars(t)
        self.update_cars(t)

In [9]:
class Graphics():
    def __init__(self):
        self.screen = pygame.display.set_mode((H, H))

    def center(self, points):
        return np.array(points) + H/2
    
    def draw_car(self, car):
        p, l, w, a = car.position(), car.length, CAR_WIDTH, car.angle()
        corners = [[-l/2,w/2], [l/2,w/2], [l/2,-w/2], [-l/2,-w/2]]

        for c in corners:
            temp =  c[0]*np.cos(a) - c[1]*np.sin(a) + p[0]
            c[1] = -c[0]*np.sin(a) - c[1]*np.cos(a) + p[1]
            c[0] = temp

        pygame.draw.polygon(self.screen, car.color, self.center(corners))
        
    def draw_stats(self, intersection, t):
        font = pygame.font.SysFont('Helvetica', 18)
        current_second = round(t/60, 1)
        
        text_time =  font.render('Seconds past: ' + str(current_second), False, (0, 0, 0))
        text_count = font.render('Number of cars: ' + str(intersection.car_count) + ' cars', False, (0, 0, 0))
        text_cars_per_minute = font.render('Cars per minute: ' + str(round(intersection.cars_per_min, 2)) + ' cars', False, (0, 0, 0))
        text_average_travel_time = font.render('Average travel time: ' + str(round(intersection.average_travel_time, 2))+' sec', False, (0, 0, 0))
        
        self.screen.blit(text_time,(20,20))
        self.screen.blit(text_count,(20,40))
        self.screen.blit(text_cars_per_minute,(20,60))
        self.screen.blit(text_average_travel_time,(20,80))

    def draw_sim(self, intersection, button_data, text_data, t):
        
        # Draw Background:
        self.screen.blit(intersection.background, (0, 0))
        
        # Draw Buttons:
        buttons = button_data.copy()
        
        for name in button_data:
            buttons[name] = pygame.draw.rect(self.screen, *buttons[name])

        for text, size, x, y in text_data:
            font = pygame.font.Font('freesansbold.ttf', size)
            text = font.render(text, True, WHITE)
            rect = text.get_rect(center=(x,y))
            self.screen.blit(text, rect)
            
        # Draw Cars:
        for car in intersection.cars():
            self.draw_car(car)

        self.draw_stats(intersection, t)
        
        pygame.display.flip()
        return buttons
        
    def draw_menu(self, button_data, text_data):
        buttons = button_data.copy()
        self.screen.fill((100,100,100))

        for name in button_data:
            buttons[name] = pygame.draw.rect(self.screen, *buttons[name])

        for text, size, x, y in text_data:
            font = pygame.font.Font('freesansbold.ttf', size)
            text = font.render(text, True, WHITE)
            rect = text.get_rect(center=(x,y))
            self.screen.blit(text, rect)

        pygame.display.flip()
        return buttons

In [10]:
class Simulation():
    def __init__(self, intersection, clock=None, graphics=None):
        self.intersection = intersection
        self.clock = clock
        self.graphics = graphics

        self.buttons = {'reset': (RED1, (740, 20, 100, 40)), 'quit': (RED1, (740, 80, 100, 40))}
        self.text = (('Reset', 21, 790, 40), ('Quit', 21, 790, 100))
        
    def run(self):
        buttons = self.graphics.draw_sim(self.intersection, self.buttons, self.text, 0)
        
        for t in itertools.count():
            self.clock.tick(FPS)
            self.intersection.update(t)
            self.graphics.draw_sim(self.intersection, self.buttons, self.text, t)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == LEFT:
                    for name in buttons: 
                        if buttons[name].collidepoint(event.pos):
                            return name
                        
    def run2(self, ticks):
        for t in range(ticks):
            self.intersection.update(t)

In [11]:
class Menu():
    def __init__(self, designs, clock, graphics):
        self.designs = designs
        self.clock = clock
        self.graphics = graphics
        self.i = 0
        
        self.buttons = {'start': (RED1, (250,250,400,40)),    'design': (RED1, (290, 310, 320, 40)),
                        'prev':  (RED2, (250, 310, 40, 40)),  'next':   (RED2, (610, 310, 40, 40)),
                        'about': (RED1, (250, 370, 400, 40)), 'quit':   (RED1, (250, 430, 400, 40))}
        
        self.text =   (('Traffic Flow', 64, 450, 100), ('By Ravi & Jelle', 21, 450, 150),
                       ('Start', 21, 450, 270),        ['', 21, 450, 330], 
                       ('Prev', 16, 269, 331),         ('Next', 16, 629, 331), 
                       ('About', 21, 450, 390),        ('Quit', 21, 450, 450))
        
    def update(self):
        self.text[3][0] = 'Design: ' + self.designs[self.i].__name__.replace('_',' ')
        return self.graphics.draw_menu(self.buttons, self.text)
        
    def run(self):
        buttons = self.update()
        
        while True:
            self.clock.tick(FPS)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == LEFT:
                    for name in buttons: 
                        if buttons[name].collidepoint(event.pos):
                            if name == 'start': 
                                return self.designs[self.i]
                            elif name == 'prev':
                                self.i = max(0, self.i - 1)
                                self.update()
                            elif name == 'next':
                                self.i = min(len(self.designs) - 1, self.i + 1)
                                self.update()
                            elif name == 'quit':
                                pygame.quit()
                                sys.exit()

In [12]:
def basic_intersection(blockades=tuple([[]]*12)):
    sections = [Line(L/2, H/2, 0, -1, H/2-L-B),
                Line(L/2, L+B, 0, -1, B),
                Line(L/2, L, 0, -1, L),
                Line(L/2, 0, 0, -1, L),
                Line(L/2, -L, 0, -1, H/2-L),
                Curve(L, L, 2, -1, 1, L/2),
                Curve(0, 0, 0, 1, 1, L/2)]
    
    for _ in range(3):
        for s in sections[-7:]:
            if isinstance(s, Line):
                sections.append(Line(s.y,-s.x,s.b,-s.a,s.length))
            else:
                sections.append(Curve(s.y,-s.x,s.a+1,s.b,s.l,s.r))
    
    l = ((0,1,2,3,4), (0,1,5,25), (0,1,2,6,10,11))
    paths = [[sections[(n+i*7)%28] for n in p] for i in range(4) for p in l]
    triggers = [sections[n*7+1] for n in range(4)]
    paths = list(zip(paths, [[triggers[n] for n in t] for t in blockades]))
    return sections, paths, triggers

In [13]:
def Free_crossing():
    blockades = ([1],[],[1,2],[2],[],[2,3],[3],[],[3,0],[0],[],[0,1])
    return basic_intersection(blockades)

In [14]:
def Shark_teeth():
    blockades = ([1,3],[3],[1,2,3],[],[],[3],[3,1],[1],[3,0,1],[],[],[1])
    return basic_intersection(blockades)

In [15]:
def Traffic_lights():
    return basic_intersection() + (True,)

In [16]:
def Roundabout():
    sections = [Line(25, 450, 0, -1, 100),
                Line(25, 350, 0, -1, 200),
                Curve(100, 150, 2, -1, 0.63, 75),
                Curve(0, 0, 3.37, 1, 0.25, 105),
                Curve(0, 0, 3.63, 1, 0.75, 105),
                Curve(150, 100, 1.63, -1, 0.63, 75),
                Line(150, 25, 1, 0, 300)]
    
    for _ in range(3):
        for s in sections[-7:]:
            if isinstance(s, Line):
                sections.append(Line(s.y,-s.x,s.b,-s.a,s.length))
            else:
                sections.append(Curve(s.y,-s.x,s.a+1,s.b,s.l,s.r))
    
    l = ((0,1,2,3,5,6),(0,1,2,3,4,10,12,13),(0,1,2,3,4,10,11,17,19,20))
    paths = [[sections[(n+i*7)%28] for n in p] for i in range(4) for p in l]
    blockades = ([[18,23,24,25]]*3 + [[25,2,3,4]]*3 + [[4,9,10,11]]*3 + [[11,16,17,18]]*3)
    paths = list(zip(paths, [[sections[n] for n in b] for b in blockades]))
    return sections, paths, []

In [17]:
def data_stats(data):
    
    travel_times_A = [value[1] for value in data[0]]    
    car_rates_A = [value[1] for value in data[1]]    
    
    # Test:
    travel_times_B = [value[1]*1.2 for value in data[0]]    
    car_rates_B = [value[1]*1.2 for value in data[1]]    
    
    travel_times_T = stats.ttest_ind(travel_times_A, travel_times_B)
    car_rates_T = stats.ttest_ind(car_rates_A, car_rates_B)
    
    print('Max travel time: ',  np.amax(travel_times_A))
    print('Mean Average travel time: ', np.mean(travel_times_A))
    print('Max car rate: ',  np.amax(car_rates_A))
    print('Mean Average cars per minute: ', np.mean(car_rates_A))
    print('')
    print('Student T test:')
    print('T travel: ', travel_times_T)
    print('T cars: ', car_rates_T)

In [18]:
pygame.init()
designs = [Free_crossing, Shark_teeth, Traffic_lights, Roundabout]

## Visualization
Run this cell to see a visualization of the simulation:

In [19]:
pygame.display.set_caption('Traffic Flow')
clock = pygame.time.Clock()
graphics = Graphics()

while True:
    menu = Menu(designs, clock, graphics)
    design = menu.run()
    
    while True:
        intersection = Intersection(design.__name__, *design())
        simulation = Simulation(intersection, clock, graphics)
        if simulation.run() == 'quit':
            break

SystemExit: 

/home/ravi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Graphs
Run this cell to generate the graphs:

In [ ]:
for attribute, label in [('cars_per_min','Output (cars/min)'), ('average_travel_time','Travel time (sec)')]:
    plt.figure(figsize=(12, 12))
    x = range(5, 120, 5)

    for design in designs:
        y = []

        for rate in x:
            intersection = Intersection(None, *design(), spawn_rate=rate)
            simulation = Simulation(intersection)
            simulation.run2(10000)
            print(intersection.cars_per_min, intersection.average_travel_time)
            y.append(getattr(intersection, attribute))

        plt.plot(x, y, label=design.__name__.replace('_',' '))

    plt.xlabel('Input (cars/min)')
    plt.ylabel(label)
    plt.legend()
#     plt.savefig('graph.png')
    plt.show()